In [123]:
import enum
import numpy as np

In [80]:
def rec_easy(arr, av, to_place):
    if arr[to_place] != 0:
        return arr
    
    sits = []
    def copy_add(arr, el, idx, where, avail):
        arr_el = arr.copy()
        arr_el[idx] = el
        ret = rec_easy(arr_el, avail, (idx+1)%len(arr_el))
        if isinstance(ret, list) and len(ret)>0:
            if isinstance(ret[0], int):
                where.append(ret)
            elif isinstance(ret[0], list):
                where += ret
    
    for i, el in enumerate(av):
        if el != to_place:
            copy_add(arr, el, to_place, sits, av[:i]+av[i+1:])
    
    return sits
    
    

In [84]:
arr = [0]*4
avail = [1,2,2,3]

A = rec_easy(arr, avail, 0)
A.sort()
A

[[1, 2, 3, 2],
 [1, 2, 3, 2],
 [2, 2, 3, 1],
 [2, 2, 3, 1],
 [2, 3, 1, 2],
 [2, 3, 1, 2],
 [3, 2, 1, 2],
 [3, 2, 1, 2]]

In [85]:
# planet X - 'X'
# comet - 'C'
# dwarf planet - 'D'
# nebula - 'N'
# asteroid - 'A'
# empty sector -'E'

class objects(enum.IntEnum):
    not_chosen = 0
    X = 1
    C = 2
    D = 3
    N = 4
    A = 5
    E = 6
    
comet_available = [1,2,4,6,10,12,16] # basic and expert are same

In [109]:
def check_pos_basic(position):
    '''
    Checks if the board is correct (only basic version)
    return 
    - True if the board is correct
    - False if something on the board is wrong
    '''
    
    for i, el in enumerate(position):
        prev_idx = (i-1)%12
        next_idx = (i+1)%12
        
        if el == objects.X:
            if position[prev_idx] == objects.D or position[next_idx] == objects.D:                
                return False
                
        elif el == objects.C:
            if i not in comet_available:
                return False      

        elif el == objects.N:
            if position[prev_idx] != objects.E and position[next_idx] != objects.E:
                return False

        elif el == objects.A:
            if position[prev_idx] != objects.A and position[next_idx] != objects.A:
                return False
            
    return True

def recursive_put(cur_pos, available, to_place):
    '''
    recursive function to get all board positions
    only 12 sectors for now
    '''
    if cur_pos[to_place] != objects.not_chosen:
        if check_pos_basic(cur_pos):
            return cur_pos
    
    situations = []
    def copy_and_add(cur_pos, obj, to_place, available, where):
        pos = cur_pos.copy()
        pos[to_place] = obj
        ret = recursive_put(pos,available, (to_place+1)%len(pos))
        
        if isinstance(ret, list) and len(ret)>0:
            if isinstance(ret[0], int):
                where.append(ret)
            elif isinstance(ret[0], list):
                where += ret      
    
    pprev_idx = (to_place-2)%12
    prev_idx = (to_place-1)%12
    next_idx = (to_place+1)%12
    
    if cur_pos[prev_idx] == objects.A and cur_pos[pprev_idx] != objects.A:
        if objects.A in available:
            idx = available.index(objects.A)
            copy_and_add(cur_pos, objects.A, to_place, available[:idx]+available[idx+1:], situations)
        else:
            return -1
    
    elif cur_pos[prev_idx] == objects.N and cur_pos[pprev_idx] != objects.E:
        if objects.E in available:
            idx = available.index(objects.E)
            copy_and_add(cur_pos, objects.E, to_place, available[:idx]+available[idx+1:], situations)
        else:
            return -1
        
    else:
        cnt = 0
        for i, el in enumerate(available):
            if el == objects.X:
                if cur_pos[prev_idx] != objects.D and cur_pos[next_idx] != objects.D:
                    copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)

            elif el == objects.C:
                if to_place in comet_available:
                    cnt +=1
                    copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)

            elif el == objects.D:
                if cur_pos[prev_idx] != objects.X and cur_pos[next_idx] != objects.X:
                    cnt +=1
                    copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)        

            elif el == objects.N:
                cnt +=1
                copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)

            elif el == objects.A:
                cnt +=1
                copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)

            elif el == objects.E:
                cnt +=1
                copy_and_add(cur_pos, el, to_place, available[:i]+available[i+1:], situations)         

#         if cnt == 0:
#             return -1
        
    return situations

In [112]:
available = [objects.A]*4 + [objects.C]*2 + [objects.D]*1 + [objects.E]*2 + [objects.N]*2
available

[<objects.A: 5>,
 <objects.A: 5>,
 <objects.A: 5>,
 <objects.A: 5>,
 <objects.C: 2>,
 <objects.C: 2>,
 <objects.D: 3>,
 <objects.E: 6>,
 <objects.E: 6>,
 <objects.N: 4>,
 <objects.N: 4>]

In [114]:
%%time
all_basic_situations = []
for planet_X_idx in range(12):
    start_pos = [objects.not_chosen]*planet_X_idx + [objects.X] + [objects.not_chosen]*(11-planet_X_idx)
    situations_i = recursive_put(start_pos, available, (planet_X_idx+1)%12)
    all_basic_situations += situations_i
len(all_basic_situations)

Wall time: 11.1 s


213408

In [120]:
all_basic_situations = all_basic_situations
different_basic_situations = []
for el in all_basic_situations:
    if el not in different_basic_situations:
        different_basic_situations.append(el)
len(different_basic_situations)

4446

In [122]:
different_basic_situations

[[<objects.X: 1>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.D: 3>,
  <objects.C: 2>,
  <objects.N: 4>,
  <objects.E: 6>,
  <objects.N: 4>,
  <objects.C: 2>,
  <objects.E: 6>],
 [<objects.X: 1>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.D: 3>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.C: 2>,
  <objects.N: 4>,
  <objects.E: 6>,
  <objects.N: 4>,
  <objects.C: 2>,
  <objects.E: 6>],
 [<objects.X: 1>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.D: 3>,
  <objects.C: 2>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.N: 4>,
  <objects.E: 6>,
  <objects.N: 4>,
  <objects.C: 2>,
  <objects.E: 6>],
 [<objects.X: 1>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.D: 3>,
  <objects.C: 2>,
  <objects.E: 6>,
  <objects.C: 2>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.N: 4>,
  <objects.E: 6>,
  <objects.N: 4>],
 [<objects.X: 1>,
  <objects.A: 5>,
  <objects.A: 5>,
  <objects.D: 3>,
  <objects.C: 2>,
  <objects.E: 6>,
  <objects.C: 2>,
  <obj

**Вероятность найти планету X в каждом из секторов**

In [126]:
x_count = [0]*12
for el in different_basic_situations:
    x_count[el.index(objects.X)] += 1
    
for i, el in enumerate(x_count):
    p = np.round(el/len(different_basic_situations)*100, 2)
    print(f'{i+1} сектор: {p}%')

1 сектор: 7.31%
2 сектор: 8.59%
3 сектор: 3.91%
4 сектор: 14.91%
5 сектор: 1.87%
6 сектор: 17.32%
7 сектор: 2.32%
8 сектор: 14.01%
9 сектор: 2.95%
10 сектор: 12.91%
11 сектор: 4.81%
12 сектор: 9.09%
